# Run workflow in an automatic way

In the previous notebook [03-process](03-process.ipynb), we ran through the workflow in detailed steps. For daily running routines, the current notebook provides a more succinct and automatic approach to run through the pipeline using some utility functions in the workflow.

In [17]:
import os
os.chdir('..')
import numpy as np
from workflow_array_ephys.pipeline import lab, subject, session, probe, ephys

## Ingestion of subjects, sessions, probes, probe insertions

1. Fill subject and session information in files `/user_data/subjects.csv` and `/user_data/sessions.csv`
2. Run automatic scripts prepared in `workflow_array_ephys.ingest` for ingestion

In [14]:
from workflow_array_ephys.ingest import ingest_subjects, ingest_sessions
ingest_subjects()
# ingest session.Session, session.SessionDirectory, probe.Probe, ephys.ProbeInsertions
ingest_sessions()


---- Insert 1 entry(s) into subject.Subject ----


FileNotFoundError: Ephys recording data not found! Neither SpikeGLX nor OpenEphys recording files found in: subject6/session1/towersTask_g0_imec0

## [Optional] Insert new ClusteringParamSet for Kilosort

This is not needed if keep using the existing ClusteringParamSet

In [2]:
params_ks = {
    "fs": 30000,
    "fshigh": 150,
    "minfr_goodchannels": 0.1,
    "Th": [10, 4],
    "lam": 10,
    "AUCsplit": 0.9,
    "minFR": 0.02,
    "momentum": [20, 400],
    "sigmaMask": 30,
    "ThPr": 8,
    "spkTh": -6,
    "reorder": 1,
    "nskip": 25,
    "GPU": 1,
    "Nfilt": 1024,
    "nfilt_factor": 4,
    "ntbuff": 64,
    "whiteningRange": 32,
    "nSkipCov": 25,
    "scaleproc": 200,
    "nPCs": 3,
    "useRAM": 0
}
ephys.ClusteringParamSet.insert_new_params(
    'kilosort2', 0, 'Spike sorting using Kilosort2', params_ks)

## Trigger autoprocessing of the remaining ephys pipeline

In [11]:
from workflow_array_ephys import populate

In [12]:
# at this stage, populate script populates through the table upstream of ClusteringTask
populate.populate()

EphysRecording:   0%|          | 0/1 [00:00<?, ?it/s]


---- Populate ephys.EphysRecording ----


LFP:   0%|          | 0/1 [00:00<?, ?it/s]


---- Populate ephys.LFP ----


LFP: 100%|██████████| 1/1 [01:20<00:00, 80.96s/it]
Clustering: 0it [00:00, ?it/s]
CuratedClustering: 0it [00:00, ?it/s]
WaveformSet: 0it [00:00, ?it/s]


---- Populate ephys.Clustering ----

---- Populate ephys.CuratedClustering ----

---- Populate ephys.WaveformSet ----


## Insert new ClusteringTask to trigger ingestion of clustering results

To populate the rest of the tables in the workflow, an entry in the `ClusteringTask` needs to be added to trigger the ingestion of the clustering results, with the two pieces of information specified:
+ the `paramset_idx` used for the clustering job
+ the output directory storing the clustering results

In [23]:
session_key = session.Session.fetch1('KEY')
ephys.ClusteringTask.insert1(
    dict(session_key, insertion_number=0, paramset_idx=0,
         clustering_output_dir='subject6/session1/towersTask_g0_imec0'), skip_duplicates=True)

In [22]:
# run populate again for table Clustering
populate.populate()

EphysRecording: 0it [00:00, ?it/s]
LFP: 0it [00:00, ?it/s]
Clustering: 0it [00:00, ?it/s]
CuratedClustering: 0it [00:00, ?it/s]
WaveformSet: 0it [00:00, ?it/s]


---- Populate ephys.EphysRecording ----

---- Populate ephys.LFP ----

---- Populate ephys.Clustering ----

---- Populate ephys.CuratedClustering ----

---- Populate ephys.WaveformSet ----


## Insert new Curation to trigger ingestion of curated results

In [24]:
key = (ephys.ClusteringTask & session_key).fetch1('KEY')
ephys.Curation().create1_from_clustering_task(key)

In [25]:
# run populate for the rest of the tables in the workflow, takes a while
populate.populate()

EphysRecording: 0it [00:00, ?it/s]
LFP: 0it [00:00, ?it/s]
Clustering: 0it [00:00, ?it/s]
CuratedClustering:   0%|          | 0/1 [00:00<?, ?it/s]


---- Populate ephys.EphysRecording ----

---- Populate ephys.LFP ----

---- Populate ephys.Clustering ----

---- Populate ephys.CuratedClustering ----


WaveformSet:   0%|          | 0/1 [00:00<?, ?it/s]


---- Populate ephys.WaveformSet ----


WaveformSet: 100%|██████████| 1/1 [31:46<00:00, 1906.83s/it]


## Summary and next step

This notebook runs through the workflow in an automatic manner, in the next notebook [05-explore](05-explore.ipynb), we will introduce how to query, fetch and visualize the contents we ingested into the tables.